# Graphiques for article

## load data and packages

In [10]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import numpy as np
from statannotations.Annotator import Annotator

In [25]:
df = pd.read_excel("./data/df_long_format_for_analysis.xlsx",index_col=0)

In [20]:
df

,ID_Experiment,Mouse_ID,Date,Infection,Group,exp,survival_original,t_origin,Time,weight
0,ID_001,TRO-05432,2014-06-05,C. albicans,1A,1,1,9.0,Tinfection,100.0
1,ID_001,TRO-05433,2014-06-05,C. albicans,1A,1,1,9.0,Tinfection,100.0
2,ID_001,TRO-05434,2014-06-05,C. albicans,1A,1,1,9.0,Tinfection,100.0
3,ID_001,TRO-05435,2014-06-05,C. albicans,1A,1,1,6.0,Tinfection,100.0
4,ID_001,TRO-05456,2014-06-05,C. albicans,1A,1,1,7.0,Tinfection,100.0
...,...,...,...,...,...,...,...,...,...,...
35350,ID_096,TRO-028337,2023-03-03,S. pneumoniae,3,3,1,5.0,T14,NaN
35351,ID_096,TRO-028338,2023-03-03,S. pneumoniae,3,3,1,4.0,T14,NaN
35352,ID_096,TRO-028339,2023-03-03,S. pneumoniae,3,3,1,6.0,T14,NaN
35353,ID_096,TRO-028342,2023-03-03,S. pneumoniae,3,3,0,11.0,T14,NaN


### utils functions

In [23]:
def remove_outlier(df):
    dead = df[df['survival_original']==1]
    dead_listeria = dead[(dead['Infection']=='Listeria')& (dead['Time'] == 'T8')]
    dead_listeria_index = dead_listeria.dropna(subset=['weight'],inplace=False).index
    infection = df[df['Infection'].isin(["Listeria","S. pneumoniae"])]
    to_high_index = infection[(infection['weight']>112) & (infection['Time'].isin(["T1","T2","T3"]))].index
    to_low_index = infection[(infection['weight']<60)].index
    #to_remove = to_high_index + to_low_index
    weight_impossible_value = to_high_index.union(to_low_index)
    index_to_remove = dead_listeria_index.union(weight_impossible_value) 
    df_clean = df.drop(index_to_remove)
    return df_clean

### cleaning df and renaming for better visulaisation

In [28]:
df_clean = remove_outlier(df)
status_mapping = {0: 'Alive', 1: 'Dead'}
df_clean["survival_original"] = df_clean['survival_original'].replace(status_mapping)

infection_mapping = {'Listeria':"L. monocytogenes"}
df_clean['Infection'] =df_clean['Infection'].replace(infection_mapping)

df_clean.rename(columns={'survival_original':'Mice'},inplace=True)

time_rename = list(np.arange(0,15,1))
time_unique = df_clean['Time'].unique()
time_mapping = dict(zip(time_unique,time_rename))
df_clean['Time'] = df_clean['Time'].replace(time_mapping)
df_clean


,ID_Experiment,Mouse_ID,Date,Infection,Group,exp,Mice,t_origin,Time,weight
0,ID_001,TRO-05432,2014-06-05,C. albicans,1A,1,Dead,9.0,0,100.0
1,ID_001,TRO-05433,2014-06-05,C. albicans,1A,1,Dead,9.0,0,100.0
2,ID_001,TRO-05434,2014-06-05,C. albicans,1A,1,Dead,9.0,0,100.0
3,ID_001,TRO-05435,2014-06-05,C. albicans,1A,1,Dead,6.0,0,100.0
4,ID_001,TRO-05456,2014-06-05,C. albicans,1A,1,Dead,7.0,0,100.0
...,...,...,...,...,...,...,...,...,...,...
35350,ID_096,TRO-028337,2023-03-03,S. pneumoniae,3,3,Dead,5.0,14,NaN
35351,ID_096,TRO-028338,2023-03-03,S. pneumoniae,3,3,Dead,4.0,14,NaN
35352,ID_096,TRO-028339,2023-03-03,S. pneumoniae,3,3,Dead,6.0,14,NaN
35353,ID_096,TRO-028342,2023-03-03,S. pneumoniae,3,3,Alive,11.0,14,NaN


In [ ]:
#violin_plot(df,'C. albicans',show=True)
path_to_save = "./graphiques/NEW_ANALYSIS/violin/"
for infection in df['Infection'].unique():
    print(infection)
    violin_plot(df,infection=infection,max_time=10,save=True,show=False,path_to_save=path_to_save)